In [ ]:
# %%
# ==========================================================
# EDA 01 — Perfilado rápido (Bicing + Meteo + Festivos)
# Dataset: data/gold/bicing_gold_final.parquet
# ==========================================================

from __future__ import annotations

import os
from pathlib import Path
import pandas as pd
import duckdb
import pyarrow.parquet as pq
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

print("✅ Imports OK")

# %%
# ==========================================================
# Rutas del proyecto (auto-detección simple)
# ==========================================================

ROOT = Path().resolve()

# Si estás dentro de notebooks/eda, subimos 2 niveles
if not (ROOT / "data").exists() and (ROOT.parent / "data").exists():
    ROOT = ROOT.parent
if not (ROOT / "data").exists() and (ROOT.parent.parent / "data").exists():
    ROOT = ROOT.parent.parent

DATA_GOLD = ROOT / "data" / "gold"
INP = DATA_GOLD / "bicing_gold_final.parquet"

OUT_SAMPLES = DATA_GOLD / "samples"
OUT_SAMPLES.mkdir(parents=True, exist_ok=True)
SAMPLE_1M = OUT_SAMPLES / "bicing_gold_final_sample_1M.parquet"

REPORTS = ROOT / "reports"
FIGURES = REPORTS / "figures"
TABLES = REPORTS / "tables"
FIGURES.mkdir(parents=True, exist_ok=True)
TABLES.mkdir(parents=True, exist_ok=True)

print("📁 ROOT:", ROOT)
print("📥 INP :", INP)
print("📤 SAMPLE_1M:", SAMPLE_1M)
print("📊 FIGURES:", FIGURES)
print("🧾 TABLES :", TABLES)

# %%
# ==========================================================
# Comprobar que existe el parquet final
# ==========================================================

if not INP.exists():
    raise FileNotFoundError(
        "No encuentro el parquet final.\n"
        f"Ruta esperada: {INP}\n\n"
        "Solución típica:\n"
        "- abre VS Code desde la raíz del repo\n"
        "- o ajusta ROOT en esta celda"
    )

meta = pq.ParquetFile(INP).metadata
print("✅ Existe. Filas (metadata):", meta.num_rows)
print("✅ Row groups:", meta.num_row_groups)

# %%
# ==========================================================
# Crear sample 1M con DuckDB (si no existe)
# ==========================================================

def duckdb_path(p: Path) -> str:
    return str(p.resolve()).replace("\\", "/")

if not SAMPLE_1M.exists():
    print("⏳ Creando sample 1M (primera vez)...")
    con = duckdb.connect(database=":memory:")
    inp = duckdb_path(INP)
    outp = duckdb_path(SAMPLE_1M)

    con.execute(
        f"""
        COPY (
          SELECT *
          FROM read_parquet('{inp}')
          USING SAMPLE 1000000 ROWS
        )
        TO '{outp}' (FORMAT PARQUET);
        """
    )
    con.close()
    print("✅ Sample creado:", SAMPLE_1M)
else:
    print("✅ Sample ya existe:", SAMPLE_1M)

print("exists?", SAMPLE_1M.exists())
if SAMPLE_1M.exists():
    print("rows(sample metadata)=", pq.ParquetFile(SAMPLE_1M).metadata.num_rows)

# %%
# ==========================================================
# Cargar sample a pandas y checks básicos
# ==========================================================

df = pq.read_table(SAMPLE_1M).to_pandas()
print("✅ df cargado | rows=", len(df), "| cols=", df.shape[1])

if not pd.api.types.is_datetime64_any_dtype(df["time_hour"]):
    df["time_hour"] = pd.to_datetime(df["time_hour"], errors="coerce")

rows = len(df)
cols = df.shape[1]
min_time = df["time_hour"].min()
max_time = df["time_hour"].max()
dup_keys = df.duplicated(["station_id", "time_hour"]).sum()

nulls = df.isna().sum().sort_values(ascending=False)
top_nulls = nulls.head(15)

print("Rango time_hour:", min_time, "->", max_time)
print("Duplicados (station_id+time_hour) en sample:", int(dup_keys))
print("\nTop nulos:\n", top_nulls)

df.head(3)

# %%
# ==========================================================
# Figuras básicas (guardadas en reports/figures/)
# ==========================================================

# 1) Histograma bikes_available_mean
fig_path = FIGURES / "eda_hist_bikes_available_mean.png"
plt.figure()
df["bikes_available_mean"].clip(lower=0).plot(kind="hist", bins=60)
plt.title("Distribución: bikes_available_mean (sample)")
plt.xlabel("bikes_available_mean")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(fig_path, dpi=140)
plt.close()
print("✅ guardado:", fig_path)

# %%
# 2) Scatter: temperatura vs bikes (subsample)
fig_path = FIGURES / "eda_scatter_temp_vs_bikes.png"

sub = df[["temperature_2m", "bikes_available_mean"]].dropna()
if len(sub) > 80000:
    sub = sub.sample(80000, random_state=42)

plt.figure()
plt.scatter(sub["temperature_2m"], sub["bikes_available_mean"], s=4)
plt.title("Temperatura vs bikes_available_mean (subsample)")
plt.xlabel("temperature_2m")
plt.ylabel("bikes_available_mean")
plt.tight_layout()
plt.savefig(fig_path, dpi=140)
plt.close()
print("✅ guardado:", fig_path)

# %%
# 3) Media bikes por hora
fig_path = FIGURES / "eda_line_mean_bikes_by_hour.png"

by_hour = df.groupby("hour", as_index=False)["bikes_available_mean"].mean()

plt.figure()
plt.plot(by_hour["hour"], by_hour["bikes_available_mean"])
plt.title("Media bikes_available_mean por hora del día")
plt.xlabel("hour")
plt.ylabel("mean bikes_available_mean")
plt.xticks(range(0, 24, 2))
plt.tight_layout()
plt.savefig(fig_path, dpi=140)
plt.close()
print("✅ guardado:", fig_path)

by_hour.head()

# %%
# 4) Media bikes por dayofweek
fig_path = FIGURES / "eda_bar_mean_bikes_by_dayofweek.png"

by_dow = df.groupby("dayofweek", as_index=False)["bikes_available_mean"].mean()

plt.figure()
plt.bar(by_dow["dayofweek"].astype(int), by_dow["bikes_available_mean"])
plt.title("Media bikes_available_mean por día semana (0=Lun..6=Dom)")
plt.xlabel("dayofweek")
plt.ylabel("mean bikes_available_mean")
plt.xticks(range(0, 7))
plt.tight_layout()
plt.savefig(fig_path, dpi=140)
plt.close()
print("✅ guardado:", fig_path)

by_dow

# %%
# 5) Boxplot festivo vs no festivo
fig_path = FIGURES / "eda_box_bikes_holiday_vs_not.png"

tmp = df[["is_holiday", "bikes_available_mean"]].dropna()

plt.figure()
tmp.boxplot(column="bikes_available_mean", by="is_holiday")
plt.suptitle("")
plt.title("bikes_available_mean: festivo(1) vs no festivo(0)")
plt.xlabel("is_holiday")
plt.ylabel("bikes_available_mean")
plt.tight_layout()
plt.savefig(fig_path, dpi=140)
plt.close()
print("✅ guardado:", fig_path)

# %%
# ==========================================================
# Correlaciones rápidas (para ideas de features)
# ==========================================================

num_cols = [
    "bikes_available_mean",
    "docks_available_mean",
    "mechanical_mean",
    "ebike_mean",
    "temperature_2m",
    "relative_humidity_2m",
    "precipitation",
    "wind_speed_10m",
    "pressure_msl",
    "lag_1h_bikes",
    "lag_24h_bikes",
    "roll3h_bikes_mean",
]
num_cols = [c for c in num_cols if c in df.columns]

corr = df[num_cols].corr(numeric_only=True)
corr_path = TABLES / "eda_corr_matrix.csv"
corr.to_csv(corr_path, index=True)

print("✅ corr guardada:", corr_path)
corr

# %%
# ==========================================================
# Estaciones top (media y variabilidad)
# ==========================================================

station_stats = (
    df.groupby("station_id")
    .agg(
        mean_bikes=("bikes_available_mean", "mean"),
        std_bikes=("bikes_available_mean", "std"),
        p95_bikes=("bikes_available_mean", lambda s: s.quantile(0.95)),
        n=("bikes_available_mean", "size"),
    )
    .reset_index()
)

top_mean = station_stats.sort_values("mean_bikes", ascending=False).head(20)
top_std = station_stats.sort_values("std_bikes", ascending=False).head(20)

top_mean_path = TABLES / "eda_top20_station_mean.csv"
top_std_path = TABLES / "eda_top20_station_std.csv"
top_mean.to_csv(top_mean_path, index=False)
top_std.to_csv(top_std_path, index=False)

print("✅ guardado:", top_mean_path)
print("✅ guardado:", top_std_path)

top_mean

# %%
top_std

# %%
# ==========================================================
# Informe Markdown (para README / landing)
# ==========================================================

report_md = []
report_md.append("# EDA Quick Profile\n")
report_md.append(f"- Filas (sample): {rows:,}\n")
report_md.append(f"- Columnas: {cols}\n")
report_md.append(f"- Rango time_hour: {min_time} -> {max_time}\n")
report_md.append(f"- Duplicados (station_id+time_hour) en sample: {int(dup_keys)}\n")

report_md.append("\n## Nulos (top 15)\n")
try:
    report_md.append(top_nulls.to_frame("null_count").to_markdown())
except Exception:
    report_md.append(top_nulls.to_string())

report_md.append("\n\n## Figuras generadas\n")
for name in [
    "eda_hist_bikes_available_mean.png",
    "eda_scatter_temp_vs_bikes.png",
    "eda_line_mean_bikes_by_hour.png",
    "eda_bar_mean_bikes_by_dayofweek.png",
    "eda_box_bikes_holiday_vs_not.png",
]:
    report_md.append(f"- reports/figures/{name}\n")

out_report = TABLES / "eda_quick_profile.md"
out_report.write_text("\n".join(report_md), encoding="utf-8")

print("✅ Report guardado:", out_report)
print(out_report.read_text(encoding="utf-8")[:700], "...\n")
